#### Bollywood 2016 reviews, 3200 reviews (1-6 negative, 7-10 positive)
preprocessing- std tokenization, Porter Stemmer, Word net lemmatizer, Remove Stop words, POS-tagger

SVM classifier (std parameters), single fold- test split 20%

#### Results: 
accuracy: 83.04821150855366
pos precision: 0.8701594533029613
pos recall: 0.880184331797235
pos F-measure: 0.8751431844215349
neg precision: 0.7450980392156863
neg recall: 0.7272727272727273
neg F-measure: 0.7360774818401937


In [2]:
import pandas as pd
import nltk
import sklearn as sk
import numpy as np
from nltk.tokenize import word_tokenize

In [86]:
from nltk.corpus import stopwords
stop=stopwords.words('english')
import re

In [4]:
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from  sklearn.base  import BaseEstimator, TransformerMixin

### READ DATA FILE

In [6]:
data=pd.read_excel('C:\\Users\\Ammick\\Google Drive\\PHD\\Research Paper\\PYTHON\\2016 Movies Reviews\\All Data + Extraction Code\\Paper II Working\\Review_Extract_Label_Bollywood.xlsx')

In [7]:
df=data.copy()

#### POLARITY ADJUSTMENT

In [8]:
df['Polarity']=df['Polarity'].replace(['Very Postive','Postive','Neutral','Very Negative'],['Positive','Positive','Negative','Negative'])

#### TRAIN TEST SPLIT 

In [111]:
pos, neg=df[df.Polarity=='Positive'].count(), df[df.Polarity=='Negative'].count()
X_train, X_test, y_train, y_test=train_test_split(df["Reviews"].values,df["Polarity"].values, test_size=0.2)

#### TOKENIZE

In [72]:
class tokenize(BaseEstimator, TransformerMixin) :
    def __init__(self,X=True):
        self.X=X
    def fit(self):
        return self.X
    def transform(self):
        y=[word_tokenize(reviews) for reviews in self.X]#['Reviews'].apply(lambda token : word_tokenize(token))
        self.X=y
        return self.X

In [112]:
    
newdf=tokenize(X_train)
X_train=newdf.transform()

#### REMOVE STOP WORDS

In [113]:
class token_stop(BaseEstimator, TransformerMixin):
    def __init__(self,X):
        self.X=X
        return None
    def fit(self):
        return self.X
    def transform(self):
        reviews=[]
        for review in self.X:
            token_stops=[w.lower() for w in review if w not in stop]
            reviews.append(token_stops)
        return reviews
    
newdf=token_stop(X_train)
X_train=newdf.transform()

#### STEMMING

In [114]:
class token_stem(BaseEstimator, TransformerMixin):
    def __init__(self, X):
        self.X=X
    def fit(self):
        return self.X
    def transform(self):
        reviews=[]
        for review in self.X:
            token_stems=[PorterStemmer().stem(w) for w in review]
            reviews.append(token_stems)
        return reviews

newdf=token_stem(X_train)
X_train=newdf.transform()

#### LEMMATIZING

In [95]:
class token_lemma(BaseEstimator, TransformerMixin):
    def __init__(self, X):
        self.X=X
    def fit(self):
        return self.X
    def transform(self):
        reviews=[]
        for review in self.X:
            token_lemmas=[WordNetLemmatizer().lemmatize(w) for w in review]
            reviews.append(token_lemmas)
        return reviews


newdf=token_lemma(X_train)
X_train=newdf.transform()

#### POS_TAGGING

In [97]:
class pos_token(BaseEstimator, TransformerMixin):
    def __init__(self, X):
        self.X=X
    def fit(self):
        return self.X
    def transform(self):
        reviews=[]
        for review in self.X:
            pos_tokens=[nltk.pos_tag(w) for w in review]
            reviews.append(pos_tokens)
        return reviews


newdf=pos_token(X_train)
X_train=newdf.transform()

In [ ]:
class data_feature(BaseEstimator, TransformerMixin):
    def __init__(self, X,y):
        self.X=X
        self.y=y
    def fit(self):
        return self.X, self.y
    def transform(self):
        newdict={}
        featureset=[]
        for a,b in zip(self.X, self.y):
            for word in set(a):
                newdict[word]=True
            featureset.append((newdict,b))  
        return featureset

    
finaltr=data_feature(X_train, y_train)
train_feature_label=finaltr.transform()

In [157]:
test_feature_label[12]

({'9-member': True,
  'neck': True,
  'war-zone.th': True,
  'hr': True,
  'blow': True,
  'expat': True,
  'mistakenli': True,
  'pre-written': True,
  'shuck': True,
  'flay': True,
  'bhatija': True,
  'brief': True,
  'kashmir': True,
  'seduct': True,
  "love'": True,
  'under-util': True,
  'gaddar': True,
  'offens': True,
  'gumption': True,
  'occup': True,
  'master-piec': True,
  'common-man': True,
  'bliss': True,
  'grand-daught': True,
  'rot': True,
  'therapist': True,
  'scintil': True,
  'hariyana.h': True,
  'madaarip.': True,
  'began.if': True,
  'robert': True,
  'weather': True,
  'blunt': True,
  'with': True,
  'them.it': True,
  'on-': True,
  'it.th': True,
  'migrat': True,
  'franz': True,
  'thrilling.3': True,
  'complex': True,
  'forgiv': True,
  'shred': True,
  '*i': True,
  'obscen': True,
  'confound': True,
  'chakrabarti': True,
  'ramanna': True,
  'serene.now': True,
  'impeach': True,
  'feel.3': True,
  'actors.storylin': True,
  'score.th': 

### CLASSIFICATION

#### NAIVE BAYES

In [ ]:
from nltk.classify import NaiveBayesClassifier
import nltk.classify.util
from nltk import precision, recall, f_measure
from sklearn.model_selection import GridSearchCV

#### SVM CLASSIFIER

In [100]:
import nltk.classify
from sklearn.svm import LinearSVC

In [102]:
svmClassifier=nltk.classify.SklearnClassifier(LinearSVC())

In [146]:
#nbClassifier=NaiveBayesClassifier()
param_grid=[{'C':[1,2,3]}]
grid=GridSearchCV(svmClassifier, param_grid, cv=3, scoring="accuracy")
svmClassifier.train(train_feature_label)



<SklearnClassifier(LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))>

In [ ]:
#Transforming Test Set


In [147]:
newdf=tokenize(X_test)
X_test=newdf.transform()
newdf=token_stop(X_test)
X_test=newdf.transform()
newdf=token_stem(X_test)
X_test=newdf.transform()
finaltr=data_feature(X_test, y_test)
test_feature_label=finaltr.transform()

### TESTING CLASSIFIER WITH METRICES

In [163]:
y_predict=[svmClassifier.classify(review) for i,(review,label) in enumerate(test_feature_label)]

In [164]:
y_predict

['Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',

In [148]:
import collections
refsets = collections.defaultdict(set)
testsets = collections.defaultdict(set)
 
for i, (feats, label) in enumerate(test_feature_label):
    refsets[label].add(i)
    observed = svmClassifier.classify(feats)
    testsets[observed].add(i)

In [150]:
print ('accuracy:', nltk.classify.util.accuracy(svmClassifier, test_feature_label)*100)
print ('pos precision:', precision(refsets['Positive'], testsets['Positive']))
print ('pos recall:', recall(refsets['Positive'], testsets['Positive']))
print ('pos F-measure:', f_measure(refsets['Positive'], testsets['Positive']))
print ('neg precision:', precision(refsets['Negative'], testsets['Negative']))
print ('neg recall:', recall(refsets['Negative'], testsets['Negative']))
print ('neg F-measure:', f_measure(refsets['Negative'], testsets['Negative']))

accuracy: 66.87402799377917
pos precision: 0.6687402799377916
pos recall: 1.0
pos F-measure: 0.8014911463187325
neg precision: None
neg recall: 0.0
neg F-measure: None
